<a href="https://colab.research.google.com/github/Guilli12pm/ML_proj/blob/master/project_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Machine Learning project

In [13]:
!git clone https://github.com/Guilli12pm/ML_proj

Cloning into 'ML_proj'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (258/258), done.
remote: Total 289 (delta 77), reused 224 (delta 27), pack-reused 0
Receiving objects: 100% (289/289), 20.68 MiB | 31.05 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [14]:
!ls ML_proj/

check_photos.py			     images.ipynb
create_data.py			     next_ite
Database			     next_ite.tar.gz
eyes_check.py			     project_ML.ipynb
haarcascade_eye_tree_eyeglasses.xml  __pycache__
haarcascade_eye.xml		     README.md
haarcascade_frontalface_default.xml  reset.py
haarcascade_smile.xml


In [15]:
!ls ML_proj/next_ite/

Guillaume_Loranchet


In [16]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()
tf.__version__

'1.13.1'

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [18]:
import pathlib
data_root_orig = tf.keras.utils.get_file('next_ite.tar.gz',
                                         'https://github.com/Guilli12pm/ML_proj/raw/master/next_ite.tar.gz',
                                         extract=True)
data_root = pathlib.Path(data_root_orig)
print(data_root)

/root/.keras/datasets/next_ite.tar.gz


In [19]:
for item in data_root.iterdir():
  print(item)

NotADirectoryError: ignored